#  Setup stuff: Connectivity

In [ ]:
# import os

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)

In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:
#  Set the display width ..

import pandas as pd
   #
pd.set_option("display.width", 480)


print("--")


# Create: Vertices/nodes, edges ..

In [ ]:

#  Vertices/Nodes ..

df_orders        = pd.DataFrame([
       #
   [101  , "Rayees Order"        ,    0,  "Order"     ],
   [102  , "Andrews Order"       ,    0,  "Order"     ],
   [103  , "Gregs Order"         ,    0,  "Order"     ],
       #
   ], columns = ["order_num", "order_name", "total", "LABEL"])

display(df_orders.head(20))

   ###
    
df_products      = pd.DataFrame([
       #
   [401  , "Apples"              ,   2,  "Product"   ],
   [402  , "Carrots"             ,  15,  "Product"   ],
   [403  , "Pears"               ,   7,  "Product"   ],
       #
   ], columns = ["product_num", "product_name", "unit_price", "LABEL"])

display(df_products.head(20))


In [ ]:

#  Create nodes ..
#
# import contextlib

for l_each in df_orders.iterrows():
    
   l_order_num    = l_each[1][0]
   l_order_name   = l_each[1][1]
   l_total        = l_each[1][2]
   l_label        = l_each[1][3]
    
   l_query = """

      CREATE ( n: {3} {{ order_num: {0} }} )                  //  Notice double curly braces
      SET n.order_name = '{1}' , n.total = {2}
   
      """.format(l_order_num, l_order_name, l_total, l_label)

   l_result0 = my_graph.query(l_query)

      ###
      ###
        
for l_each in df_products.iterrows():
    
   l_product_num  = l_each[1][0]
   l_product_name = l_each[1][1]
   l_unit_price   = l_each[1][2]
   l_label        = l_each[1][3]
    
   l_query = """

      CREATE ( n: {3} {{ product_num: {0} }} ) 
      SET n.product_name = '{1}' , n.unit_price = {2}
   
      """.format(l_product_num, l_product_name, l_unit_price, l_label)

   l_result0 = my_graph.query(l_query)

print("--")


In [ ]:
#  What's in the vertices/nodes ..

l_result = my_graph.query("""

   MATCH ( n ) 
   RETURN n
   
   """)

print(l_result[0:30])


# ##################################################################

In [ ]:
#  Create the edge ..

df_purchased = pd.DataFrame([
      #
   [ 101, 401 ,"CONTAINS"],
   [ 101, 402 ,"CONTAINS"],
   [ 101, 403 ,"CONTAINS"],
      #
   [ 102, 401 ,"CONTAINS"],
   [ 102, 401 ,"CONTAINS"],
   [ 102, 403 ,"CONTAINS"],
      #
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
   [ 103, 403 ,"CONTAINS"],
      #
   ], columns = ["START_ID", "END_ID", "TYPE"])

df_purchased.head(30)


In [ ]:

for l_each in df_purchased.iterrows():
    
   l_startid    = l_each[1][0]
   l_endid      = l_each[1][1]
   l_type       = l_each[1][2]
    
   l_query = """
      
      MATCH (n: Order  ) WHERE n.order_num   = {0}
      MATCH (m: Product) WHERE m.product_num = {1}
      CREATE (n) -[r: {2} ]-> (m)
      
      """.format(l_startid, l_endid, l_type)
    
   l_result0 = my_graph.query(l_query)

print("--")


In [ ]:
#  What's in the graph .. 

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


#  Finally now; Why we're here ..

In [ ]:
#  The traversal we are trying to run ..

l_result = my_graph.query("""

   //  Just fetching, works
   //
   MATCH (n: Order) - [r :CONTAINS ] -> (m: Product)
   RETURN n, sum(m.unit_price) AS x_cost
   
   //  Closest to original question, throws error
   //
   // MATCH (n: Order) - [r :CONTAINS ] -> (m: Product)
   // WITH n, sum(m.unit_price) AS x_cost
   // SET Order.total = x_cost
   //
   //  OperationError: backtrace (QueryClient.cpp:391): unable to translate openCypher parse tree to internal IR (OpGraph.cpp:226):
   //  Syntax error: The query cannot be recognized by openCypher. (CypherOpGraphBuilder.cpp:63): TCK = SyntaxError:UnexpectedSyntax
   //  Katana = SyntaxError:ParserError: TCK = SyntaxError:UnexpectedSyntax
   //  Katana = SyntaxError:ParserError
   
   //  This also errors ..
   //
   // MATCH (n: Order) - [r :CONTAINS ] -> (m: Product)
   // WITH sum(m.unit_price) AS x_cost
   // SET Order.total = x_cost
   
   """, contextualize=True)

l_result.view()


#  Exact, original syntax ..
#
#  MATCH (loan)-[:hasLoanSnapshot]->(snapshot)
#  WITH loan, sum(snapshot.loan_snapshot_flag) as loan_snapshots
#  SET loan.loan_flag = loan_snapshots

